-----

## Load package(s)

In [ ]:
from pytubefix import Playlist, YouTube

In [ ]:
yt = YouTube('http://youtube.com/watch?v=oWi8ZncK9Tw')

In [ ]:
print(yt.captions)
print(yt.title)
print(yt.publish_date)

----

## Playlist

In [ ]:
# Dialogue works
channelName = "DialogueWorks"
URL_PLAYLIST = "https://www.youtube.com/playlist?list=PLzImU_KHY9-JjAIg0AZXxHaMhJD6UXfNj"

# channelName = "NewAtlas"
# URL_PLAYLIST = "https://www.youtube.com/playlist?list=PLkhlYgGXRhhxHzOmBH10Cp0nGBWjLATjc"

# Retrieve URLs of videos from playlist
playlist = Playlist(URL_PLAYLIST)
print('Number Of Videos In playlist: %s' % len(playlist.video_urls))

urls = []
for url in playlist:
    urls.append(url)

In [ ]:
ytObjects = []
for u in urls:
    ytObjects.append(YouTube(u))

In [ ]:
titles = [obj.title for obj in ytObjects]
len(titles)

In [ ]:
focusIndexes = []
k = 0
for t in titles:
    if "Wolff" in t or 'Hundson' in t:
        focusIndexes.append(k)
    k = k + 1 

len(focusIndexes)

In [ ]:
ytObjectsFiltered = [ ytObjects[i] for i in focusIndexes ]
len(ytObjectsFiltered)

In [ ]:
ytObjectsFiltered[0].title

In [ ]:
ytObjectsFiltered[0].watch_url

In [ ]:
urlsFiltered = [ obj.watch_url for obj in ytObjectsFiltered ]
len(urlsFiltered)

In [ ]:
urlToTitleRules = {obj.watch_url.split("v=")[1] : obj.title for obj in ytObjectsFiltered}
len(urlToTitleRules)

In [ ]:
list(urlToTitleRules.items())[0:5]

-------

## Get YouTube transcripts

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(video_url):
    try:
        # Extract the video ID from the URL
        video_id = video_url.split('v=')[1]

        # Retrieve the transcript for the video
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages = ("en",))

        # Concatenate the text from each transcript segment
        transcript_text = ' '.join([segment['text'] for segment in transcript])

        return transcript_text

    except Exception as e:
        print(f'An error occurred: {str(e)}')

In [ ]:
transcripts = { u : get_transcript(u) for u in urlsFiltered}
print(len(transcripts))

In [ ]:
list(transcripts.items())[0][1]

In [ ]:
list(transcripts.items())[0]

In [ ]:
docs = { u.split("v=")[1] : t for u, t in transcripts.items() }
len(docs)

------

## Export

In [ ]:
dirName = "./DialogueWorks"

In [ ]:
for p in list(docs.items()):
    fileName = dirName + "/" + p[0] + ".txt"
    with open(fileName, 'w') as f:
        f.write(p[1])